# **Install env & package**

```python
conda create --name yourenv python=3.5 anaconda
source activate tensorflow
conda install -c conda-forge jupyterlab
pip install --ignore-installed --upgrade tensorflow==1.4.1
```

In [2]:
import tensorflow as tf
tf.__version__

'1.12.0'

In [4]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True

In [6]:
config = tf.ConfigProto()


In [9]:
config.gpu_options.allow_growth

False

---

# **목표**
    - Tensorflow 그래프를 이용한 계산 이해
    - Tensorflow에서 미리 만들어진 함수를 사용
    - 모델을 생성 및 활용

---

#  **현재 사용되고 있는 딥러닝 패키지 현황(2018)**

<p align="center"><img width="500" height="auto" src="./picture/top-16-deep-learning-libs-691.jpg"></p>

---

# **Tensorflow 특징**
- Python API
- 로컬이나 서버에서 CPU, GPU를 사용하여 계산할 수 있음
- Window, Linux계열에 적용가능
- tensorboard를 활용한 시각화
- checkpoints를 활용한 실험관리
- 자동적으로 계산되는 gradient(Auto-differentiation)
- 많은 Q & A 정보

---

# **보다 텐서 플로우를 쉽게 사용하는 방법**
- TF learn, TF Slim, Pretty Tensor, **Keras**
- 모델을 이해하는데 어려운 점이 있으며, tf 패키지 업데이트가 되면서 잘 작동이 안되는 경우가 많음
- Scratch로 학습하는 것이 이해가 쉬우며, error에 대응하기가 수월함

---

# **Data(tensor) flow Graphs**
    - 단계1: 그래프를 구축
    - 단계2: 그래프 안에 있는 operation을 실행하기 위한 session을 실행

<p align="center"><img width="500" height="auto" src="./picture/data_flow.jpg"></p>

- **Tensor의 정의**
    - n-dimensional array
        - 0-d tensor: scalar(number)
        - 1-d tensor: vector
        - 2-d tensor: matrix

In [10]:
import numpy as np

In [11]:
# 0-d tensor: scalar(number)
np.random.rand(1)

array([0.97098002])

In [5]:
# 1-d tensor: vector
np.random.rand(2)

array([0.74775915, 0.15879868])

In [6]:
# 2-d tensor: matrix
np.random.rand(3,2)

array([[0.32923685, 0.51731142],
       [0.89829463, 0.18190867],
       [0.67366578, 0.31150177]])

In [7]:
# 3-d tensor: matrix
np.random.rand(2,3,2)

array([[[0.4532859 , 0.21945748],
        [0.36196948, 0.73142101],
        [0.61413488, 0.10165142]],

       [[0.54491764, 0.51150568],
        [0.75647818, 0.98904049],
        [0.13197081, 0.9494673 ]]])

In [8]:
import tensorflow as tf

```python
tf.add(x,y)
```

<p align="center"><img width="400" height="auto" src="./picture/data_flow_01.jpg"></p>

<p align="center"><img width="400" height="auto" src="./picture/data_flow_02.jpg"></p>

In [9]:
a = tf.add(3,5)
print(a)

Tensor("Add:0", shape=(), dtype=int32)


- 목적에 따라 실수 타입/정수 타입을 명확히 구분해 줘야 함
    - 손실함수
    - Masking

In [10]:
b = tf.add(3.0,5.0)
print(b)

Tensor("Add_1:0", shape=(), dtype=float32)


- **Tensor graph의 구조**
    - **Nodes**: operations($+$,$-$,$\times$,$\div$), variables(weights), constants
    - **Edges**: tensors

- Session을 실행시켜주면, 종료선언을 해 주어야 함
- 또다른 방법으로 With를 사용하여 시작과 종료를 함께 선언할 수 있음

**case 1)**

In [11]:
sess = tf.Session(config=config)

In [12]:
sess.run(a)

8

In [13]:
sess.close()

**case 2)**

In [14]:
with tf.Session(config=config) as sess:
    print(sess.run(a))

8


- **tf.Session()이 역할**
    - 어떤 operation 객체가 있는지, 어떤 tensor가 계산되는지에 대한 정보를 캡슐화
    - 텐서 그래프를 연결하거나 종료하는 것

- **more graph**

<p align="center"><img width="400" height="auto" src="./picture/data_flow_03.jpg"></p>

In [15]:
x=2
y=3
op1 = tf.add(x,y)
op2 = tf.multiply(x,y)
op3 = tf.pow(op2,op1) # op2^{op1}
print(op3)

Tensor("Pow_1:0", shape=(), dtype=int32)


In [16]:
with tf.Session(config=config) as sess:
    _op3 = sess.run(op3,feed_dict={x:6, y:23}) # Don't use the same object name

TypeError: Cannot interpret feed_dict key as Tensor: Can not convert a int into a Tensor.

In [17]:
print(_op3) # 6**5

7776


- **more sub graph**

<p align="center"><img width="400" height="auto" src="./picture/data_flow_05.jpg"></p>

In [18]:
x = 2
y = 3
op1 = tf.add(x,y)
op2 = tf.multiply(x,y)
useless = tf.multiply(x,op1)
op3 = tf.pow(op2,op1)
with tf.Session(config=config) as sess:
    _op3 = sess.run(op3)

- op3만 sess.run할 경우 관련있는 operation만 실행됨

In [19]:
print(_op3) # 6**5

7776


In [20]:
with tf.Session(config=config) as sess:
    _op3, not_useless = sess.run([op3, useless])

In [21]:
print('op3:', _op3)
print('not_useless:', not_useless)

op3: 7776
not_useless: 10


- sess.run과 관련해서 fetch와 관련된 variable를 feed_dict를 선언해 주어야 함
    - sess.run(fetch,feed_dict=None)
    - sess.run(op3,feed_dict={x:2, y:3})

---

# **Distributed Computation**

In [22]:
tf.reset_default_graph()
with tf.device('/cpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], name ='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], name ='b')
    
with tf.device('/gpu:0'):
    c = tf.multiply(a,b)

- allow_soft_placement
    - 아래의 조건이 참일 때 CPU 수행
        - OP에 대한 GPU 실행이 없을 때 
        - GPU 장치가 없거나 인지되지 않을 때
        

- log_device_placement
    - operation에 특정 tf.device를 할당할 때 사용

In [23]:
cf =tf.ConfigProto(device_count={"CPU": 1,
                                     "GPU": 1},
                       allow_soft_placement=True,
                       log_device_placement=True)
cf.gpu_options.allow_growth=True
sess = tf.Session(config = cf)

In [24]:
print(sess.run(c))

[ 1.  4.  9. 16. 25. 36.]


---

# **Multiple Graph**
- tf.Graph()
- operation이 tf.Graph에 의존적임
- Session에 출력하고자하는 op와 관련된 graph를 할당해 줘야 함

In [25]:
g = tf.Graph()
g

In [26]:
with g.as_default():
    x = tf.add(3,5)

In [27]:
with tf.Session(graph=g, config=config) as sess:
    print(sess.run(x))

8


In [28]:
with g.as_default():
    a = 3
    b = 5
    x = tf.add(a,b)

In [29]:
with tf.Session(graph=g, config=config) as sess:
    print(sess.run(x))

8


- 그래프 초기화

In [30]:
x = tf.add(3,5)
x

<tf.Tensor 'Add:0' shape=() dtype=int32>

In [31]:
tf.reset_default_graph()
x = tf.add(3,5)
x

<tf.Tensor 'Add:0' shape=() dtype=int32>

- 특정 그래프에 할당되지 않은 default graph와 created graph를 섞어서 사용하면, 코드 해석이 복잡해질 가능성이 큼

In [32]:
g =tf.Graph()

# for default graph
a = tf.constant(3)

# for created graph
with g.as_default():
    b = tf.constant(5)


In [33]:
with tf.Session(config=config) as sess:
    print(sess.run(a))

3


- 따라서 default graph를 하나의 subgraph로 선언해줘 쉽게 이해할 수 있는 코딩이 됨
- 그래프를 두개로 나눠서 할 경우, 한 그래프에서 session이 켜졌을 때, 다른 그래프는 build할 수 있음 

In [34]:
tf.reset_default_graph()

g1 = tf.get_default_graph() #get current graph
g2 = tf.Graph()

# for default graph
with g1.as_default():
    a = tf.constant(3)
# for created graph
with g2.as_default():
    b = tf.constant(5)

In [ ]:
a = tf.constant(3)
b = tf.constant(5)

In [35]:
with tf.Session(graph=g1, config=config) as sess:
    print(sess.run(a))

3


In [36]:
tf.reset_default_graph()

g1 = tf.get_default_graph() #get current graph
g2 = tf.Graph()

In [37]:
with tf.Session(graph=g1, config=config) as sess1:
    print(sess1.graph.as_graph_def())
    
with tf.Session(graph=g2, config=config) as sess2:
    print(sess2.graph.as_graph_def())

versions {
  producer: 24
}

versions {
  producer: 24
}



In [ ]:
tf

---

# **Graph의 장점**
- 필요 시 subgraph만 계산할수도 있음
- distributed computation
- 그래프 시각화
- 그래디언트를 부분적으로 쉽게 계산할 수 있음